<a href="https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/tool/SD_Converters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Connect and mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Loading files

In [ ]:
#@title Install requirements
from IPython.display import clear_output

!wget -q https://github.com/steinhaug/diffusers/raw/main/scripts/convert_original_stable_diffusion_to_diffusers.py
!wget -q https://github.com/steinhaug/diffusers/raw/main/scripts/convert_diffusers_to_original_stable_diffusion.py
%pip install -qq git+https://github.com/steinhaug/diffusers
%pip install -q -U --pre triton
%pip install omegaconf
%pip install -q accelerate ftfy bitsandbytes==0.35.0 natsort safetensors
#%pip install -q accelerate transformers ftfy bitsandbytes==0.35.0 gradio natsort safetensors xformers

!mkdir /content/download
clear_output()
print('[1;32mDone! ✓')

Done! ✓


In [ ]:
#@title huggingface_hub: Install and Connect (have API key ready)
from IPython.display import clear_output
!pip install --upgrade huggingface_hub
!mkdir /content/huggface_cache
clear_output()
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
#@title huggingface_hub: Downloading private files from huggingface
from huggingface_hub import hf_hub_download
#hf_hub_download(repo_id="steinhaug/zkiste", filename="zkiste3-5050.ckpt", cache_dir="/content/huggface_cache")


# download a file
#from huggingface_hub import hf_hub_download
#path = hf_hub_download(repo_id="steinhaug/onceUponAtimeInPornVille", repo_type="dataset", local_dir="/content/drive/MyDrive/offload")
#print(path)

# Download full repository
#from huggingface_hub import snapshot_download
#path = snapshot_download(repo_id="steinhaug/onceUponAtimeInPornVille", repo_type="dataset", local_dir="/content/drive/MyDrive/offload")


In [ ]:
%cd /content/download/
!wget online_path -O filename

## Converters

In [ ]:
#@title Convert ckpt &gt; diffusers

PATH_CKPT = "/content/huggface_cache/models--steinhaug--zkiste/snapshots/af385428edf2f108a52b4a236ae14b4780a6d5e5/zkiste3-5050.ckpt" #@param {type:"string"}
PATH_DIFFUSERS = "/content/diff_model" #@param {type:"string"}

#@markdown <em>path_ckpt:</em> path to the checkpoint file you want to convert to diffusers model<br>
#@markdown <em>path_diffusers:</em> path to the folder you want the diffusers model to be saved in, if it doesnt exist it will be created for you.

from pathlib import Path

path = Path(PATH_CKPT)
if not path.is_file():
  raise Exception(f"File not found! Path: {PATH_CKPT}")

path = Path(PATH_DIFFUSERS)
if not path.exists():
  print(f"[*] Creating directory for diffuser model...")
  path.mkdir(parents = False, exist_ok = False)

%cd /content/
!python convert_original_stable_diffusion_to_diffusers.py --scheduler_type ddim --checkpoint_path $PATH_CKPT --image_size 512 --prediction_type epsilon --dump_path $PATH_DIFFUSERS --device cpu

WEIGHTS_DIR = PATH_DIFFUSERS

In [ ]:
#@title Convert diffusers &gt; ckpt

%cd /content/

#@markdown Run conversion, saves .ckpt file in __weights_dir__
WEIGHTS_DIR = "/content/drive/MyDrive/stable_diffusion_weights/sd_helchr/2222" #@param {type:"string"}
CKPT_NAME = "model.ckpt" #@param {type:"string"}

ckpt_path = WEIGHTS_DIR + "/" + CKPT_NAME

from pathlib import Path
path = Path(WEIGHTS_DIR)
if not path.exists():
  raise Exception(f"Directory not found! Path: {WEIGHTS_DIR}")

half_arg = ""
#@markdown  Whether to convert to fp16, takes half the space (2GB).
fp16 = True #@param {type: "boolean"}
if fp16:
    half_arg = "--half"
!python convert_diffusers_to_original_stable_diffusion.py --model_path $WEIGHTS_DIR  --checkpoint_path $ckpt_path $half_arg
print(f"[*] Converted ckpt saved at {ckpt_path}")

## other tools...